<a href="https://colab.research.google.com/github/pmopedro/computational-chess/blob/fabio/chess_data_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importação de bibliotecas

In [1]:
!pip install python-chess
import chess.pgn
import io
import pandas as pd
from google.colab import drive

### Leitura dos dados de treinamento

In [2]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:


# Leitura dos dados das partidas de xadrez
train_folder = '/content/gdrive/MyDrive/IA006_Projeto/train/'
datainfo = train_folder + 'chessData.csv'

df_train = pd.read_csv(datainfo)

In [4]:
df_train

,FEN,Evaluation
0,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,-10
1,rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...,+56
2,rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...,-9
3,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...,+52
4,rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...,-26
...,...,...
12958030,r1bqkb1r/pp3ppp/1nn1p3/3pP3/3P1P2/1B3N2/PP2Q1P...,+6
12958031,r2qkb1r/pp1b1ppp/1nn1p3/3pP3/3P1P2/1B3N2/PP2Q1...,+84
12958032,r2qkb1r/pp1b1ppp/1nn1p3/3pP3/3P1P2/1BN2N2/PP2Q...,0
12958033,r2qkb1r/pp1b1ppp/1n2p3/n2pP3/3P1P2/1BN2N2/PP2Q...,+115


### Leitura dos dados de teste

In [5]:
test_folder = '/content/gdrive/MyDrive/IA006_Projeto/test/'
test_info = test_folder + 'games.csv'

df_test = pd.read_csv(test_info)

In [6]:
df_test = df_test[['id', 'turns', 'winner', 'white_rating', 'black_rating', 'moves']]

In [7]:
# Dropa jogos com menos de 22 lances.
df_test = df_test[df_test.turns >= 22]

In [8]:
df_test

,id,turns,winner,white_rating,black_rating,moves
2,mIICvQHh,61,white,1496,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...
3,kWKvrqYL,61,white,1439,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...
4,9tXo1AUZ,95,white,1523,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...
6,qwU9rasv,33,white,1520,1423,d4 d5 e4 dxe4 Nc3 Nf6 f3 exf3 Nxf3 Nc6 Bb5 a6 ...
8,dwF3DJHO,66,black,1439,1392,e4 e5 Bc4 Nc6 Nf3 Nd4 d3 Nxf3+ Qxf3 Nf6 h3 Bc5...
...,...,...,...,...,...,...
20053,EfqH7VVH,24,white,1691,1220,d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...
20054,WSJDhbPl,82,black,1233,1196,d4 d6 Bf4 e5 Bg3 Nf6 e3 exd4 exd4 d5 c3 Bd6 Bd...
20055,yrAas0Kj,35,white,1219,1286,d4 d5 Bf4 Nc6 e3 Nf6 c3 e6 Nf3 Be7 Bd3 O-O Nbd...
20056,b0v4tRyF,109,white,1360,1227,e4 d6 d4 Nf6 e5 dxe5 dxe5 Qxd1+ Kxd1 Nd5 c4 Nb...


#### Conversão de PGN para FEN na profundidade 22

In [9]:
board_position = []
for game_moves in df_test['moves']:
  pgn = io.StringIO(game_moves)
  game = chess.pgn.read_game(pgn)

  board = game.board()
  move_cnt = 0
  for move in game.mainline_moves():
    board.push(move)
    move_cnt = move_cnt + 1
    if (move_cnt == 22): break

  board_position.append(board.fen())

In [10]:
# Adiciona lista de posições no dataframe.
se = pd.Series(board_position)
df_test['position'] = se.values

In [11]:
# Dropa a coluna dos movimentos, não é mais necessária.
df_test = df_test.drop('moves', axis=1)

In [12]:
# Exporta o dataframe para um novo arquivo csv no Drive, na pasta de teste.
test_output = test_folder + 'games_positions.csv'
df_test.to_csv(test_output)